<h1><b>Valorant Data Collection

<h2><b>Imports


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

<h2><b>Getting list of top players

In [ ]:
top_players_url = []
def get_top_players_url(ep = 6, act = 3):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = f'https://blitz.gg/valorant/leaderboards/official?actName=e{ep}act{act}'
    webpage=requests.get(url ,headers=headers).text
    soup=BeautifulSoup(webpage,'lxml')
    for i in range(len(soup.find_all('div', class_ = 'â¡b73efc61 row â¡f6341061'))):
        top_players_url.append('https://blitz.gg' + soup.find_all('div', class_ = 'â¡b73efc61 row â¡f6341061')[i].find_all('a')[0].get('href').replace(' ', '%20'))

In [ ]:
get_top_players_url(6, 1)
get_top_players_url(6, 2)
get_top_players_url(6, 3)
get_top_players_url(7, 1)

In [ ]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)
for i in top_players_url:
    if is_ascii(i) == False:
        top_players_url.remove(i)
len(top_players_url)

187

<h2><b>Collecting Match URLs of Players

In [ ]:
match_url = []
def get_match_url(player_url, ep, act):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = player_url + f'?actName=e{ep}act{act}'
    webpage=requests.get(url ,headers=headers).text
    soup=BeautifulSoup(webpage,'lxml')
    no_of_matches = len(soup.find_all('a', class_ = "â¡ae84b94b"))
    for match_ in range(no_of_matches):
        match_type = soup.find_all('div', class_ = 'â¡7245df4f')[match_].find_all('p', class_ = 'type-caption queue-name')[0].text.strip()

        if match_type == 'Unrated' or match_type == 'Competitive':
            map = soup.find_all('div', class_ = 'match-kda')[match_].find_all('p', class_ = 'type-caption match-sub-stat')[0].text.strip()
            match_url.append(["https://blitz.gg" + soup.find_all('a', class_ = "â¡ae84b94b")[match_].get('href'), map])

In [ ]:
for i, player in enumerate(top_players_url[0:2]):
    get_match_url(player, 6, 1)
    get_match_url(player, 6, 2)
    get_match_url(player, 6, 3)
    get_match_url(player, 7, 1)
    print(i+1)

1
2


In [ ]:
len(match_url)

48

In [ ]:
import pickle
pickle.dump(match_url, open('match_url.pkl', 'wb'))

In [ ]:
file = open('/content/match_url.pkl', 'rb')
match_url = pickle.load(file)
file.close()
len(match_url)

In [ ]:
match_url[0][0].split('/')[-1]

In [ ]:
match_dataset = {}

<h2><b>Collecting Match Data

In [ ]:
def get_match_data(match_):
    try:
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}

        url = match_[0]
        webpage=requests.get(url ,headers=headers).text
        soup=BeautifulSoup(webpage,'lxml')
        player = soup.find_all('div', class_ = 'â¡248204a8')[0]

        # match_id = match_[0][0].split('/')[-1]
        map = match_[1]
        match_dataset[url] = [map]

        outcome = soup.find_all('h1', class_ = 'flex type-h5')[0].text.strip()

        agent = player.find_all('img')[0].get('src').replace('https://blitz-cdn.blitz.gg/blitz/val/agents/matchtile/', '').replace('-art-withBG.webp', '').replace('-art-withBG-2.webp', '')
        avg_score = int(player.find_all('div', class_ = 'col-score')[0].find_all('p')[0].text.strip())
        KDA = player.find_all('div', class_ = 'col-kda')[0].find_all('p')[0].text.strip().replace('/', '').split('  ')
        kill = int(KDA[0])
        death = int(KDA[1])
        assist = int(KDA[2])
        KD = float(kill/death)
        KD = round(KD, 2)
        Econ = int(player.find_all('div', class_ = 'col-economy')[0].find_all('p')[0].text.strip())
        match_dataset[url].append(agent)
        match_dataset[url].append(KD)
        match_dataset[url].append(Econ)
        match_dataset[url].append(avg_score)

        for player_no in range(4):
            player = soup.find_all('div', class_ = 'â¡e80812ff')[player_no]
            agent = player.find_all('img')[0].get('src').replace('https://blitz-cdn.blitz.gg/blitz/val/agents/matchtile/', '').replace('-art-withBG.webp', '').replace('-art-withBG-2.webp', '')
            avg_score = int(player.find_all('div', class_ = 'col-score')[0].find_all('p')[0].text.strip())
            KDA = player.find_all('div', class_ = 'col-kda')[0].find_all('p')[0].text.strip().replace('/', '').split('  ')
            kill = int(KDA[0])
            death = int(KDA[1])
            assist = int(KDA[2])
            KD = float(kill/death)
            KD = round(KD, 2)
            Econ = int(player.find_all('div', class_ = 'col-economy')[0].find_all('p')[0].text.strip())
            match_dataset[url].append(agent)
            match_dataset[url].append(KD)
            match_dataset[url].append(Econ)
            match_dataset[url].append(avg_score)

        for player_no in range(5):
            player = soup.find_all('div', class_ = 'â¡dbae468a')[player_no]
            agent = player.find_all('img')[0].get('src').replace('https://blitz-cdn.blitz.gg/blitz/val/agents/matchtile/', '').replace('-art-withBG.webp', '').replace('-art-withBG-2.webp', '')
            avg_score = int(player.find_all('div', class_ = 'col-score')[0].find_all('p')[0].text.strip())
            KDA = player.find_all('div', class_ = 'col-kda')[0].find_all('p')[0].text.strip().replace('/', '').split('  ')
            kill = int(KDA[0])
            death = int(KDA[1])
            assist = int(KDA[2])
            KD = float(kill/death)
            KD = round(KD, 2)
            Econ = int(player.find_all('div', class_ = 'col-economy')[0].find_all('p')[0].text.strip())
            match_dataset[url].append(agent)
            match_dataset[url].append(KD)
            match_dataset[url].append(Econ)
            match_dataset[url].append(avg_score)

        match_dataset[url].append(outcome)
    except:
        pass

In [ ]:
len(match_dataset)

0

<h3>Fetching Data in Batch


In [ ]:
for i in range(0, 1000):
    get_match_data(match_url[i])
    print(i+1)

In [ ]:
len(match_dataset)

In [ ]:
for i in range(1000, 2000):
    get_match_data(match_url[i])
    print(i+1)

In [ ]:
len(match_dataset)

In [ ]:
for i in range(2000, 3000):
    get_match_data(match_url[i])
    print(i+1)

In [ ]:
len(match_dataset)

In [ ]:
for i in range(3000, 4000):
    get_match_data(match_url[i])
    print(i+1)

In [ ]:
len(match_dataset)

In [ ]:
for i in range(4000, 5000):
    get_match_data(match_url[i])
    print(i+1)

In [ ]:
len(match_dataset)

In [ ]:
for i in range(5000, 6623):
    get_match_data(match_url[i])
    print(i+1)

In [ ]:
len(match_dataset)

In [ ]:
# for i in range(len(match_url)):
# # for i in range(10):
#     get_match_data(match_url[i])
#     print(i+1)

In [ ]:
from pprint import pprint
pprint(match_dataset)

<h2><b>Changing Collected Data Format

In [ ]:
import pandas as pd
columns = ['map', 'A_player_1_agent', 'A_player_1_KD', 'A_player_1_Econ', 'A_player_1_avg_score',
           'A_player_2_agent', 'A_player_2_KD', 'A_player_2_Econ', 'A_player_2_avg_score',
           'A_player_3_agent', 'A_player_3_KD', 'A_player_3_Econ', 'A_player_3_avg_score',
           'A_player_4_agent', 'A_player_4_KD', 'A_player_4_Econ', 'A_player_4_avg_score',
           'A_player_5_agent', 'A_player_5_KD', 'A_player_5_Econ', 'A_player_5_avg_score',
           'B_player_1_agent', 'B_player_1_KD', 'B_player_1_Econ', 'B_player_1_avg_score',
           'B_player_2_agent', 'B_player_2_KD', 'B_player_2_Econ', 'B_player_2_avg_score',
           'B_player_3_agent', 'B_player_3_KD', 'B_player_3_Econ', 'B_player_3_avg_score',
           'B_player_4_agent', 'B_player_4_KD', 'B_player_4_Econ', 'B_player_4_avg_score',
           'B_player_5_agent', 'B_player_5_KD', 'B_player_5_Econ', 'B_player_5_avg_score',
           'A_outcome']
dataset = pd.DataFrame.from_dict(match_dataset, orient='index', columns=columns).reset_index()
dataset

<h2><b>Saving Dataset

In [ ]:
pickle.dump(dataset, open('valo_dataset.pkl', 'wb'))